In [1]:
import numpy as np
from simple.hurst import HurstRS, HurstDMA, HurstDSOD
from simple.funcs import hurst
from simple.jurik import JCFB
from simple.pretty import prun
from simple.numpy import rolling_apply
import pymannkendall as mk
from statsmodels.tsa.stattools import adfuller
from simple.plotly import chartFigure, updateLines

In [2]:
period = 512

In [19]:
A = (np.random.rand(period*5)-0.4).cumsum()
B = np.random.rand(period*5) + A[-1]
T = np.hstack((A, B))
len(T)

5120

In [24]:
fig = chartFigure(Price=T, rows=5, height=800,
    HurstDSOD={'row': 2}, hurstSeries={'row': 2}, HurstRS={'row': 2, 'opacity': 0.5}, HurstDMA={'row': 2}, 
    MannKendall={'row': 3},
    CFB={'row': 4},
    ADF={'row': 5}
)
fig.update_layout(legend={'y': 1.05})
fig

FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> Price <i style="color:#fc99…

In [5]:
h = JCFB(T, 9, 36)
updateLines(fig, CFB=h)

In [6]:
def ADFSeries(X: np.array, Period: int) -> np.array:
    x = rolling_apply(lambda x: adfuller(x)[0], Period, X, n_jobs=128, progress=True)
    x[:period] = x[period]
    return x

h = ADFSeries(T, period)
updateLines(fig, ADF=h)

  0%|          | 0/4608 [00:00<?, ?it/s]

In [7]:
def mannkendallSeries(X: np.array, Period: int) -> np.array:
    x = rolling_apply(lambda x: mk.original_test(x).Tau, Period, X, n_jobs=128, progress=True)
    x[:period] = x[period]
    return x

h = mannkendallSeries(T, period)
updateLines(fig, MannKendall=h)

  0%|          | 0/4608 [00:00<?, ?it/s]

In [8]:
def hurstSeries(X: np.array, Period: int) -> np.array:
    x = rolling_apply(lambda x: hurst(x), Period, X, n_jobs=128, progress=True)
    x[:period] = x[period]
    return x

h = hurstSeries(T, period)
updateLines(fig, hurstSeries=h)

  0%|          | 0/4608 [00:00<?, ?it/s]

In [9]:
h = HurstDSOD(T, period)
updateLines(fig, HurstDSOD=h)

In [10]:
h = prun(lambda src, period: HurstRS(np.diff(src, prepend=0), period), T, period)
updateLines(fig, HurstRS=h)

  0%|          | 0/256 [00:00<?, ?it/s]

In [11]:
h = prun(HurstDMA, T, period)
updateLines(fig, HurstDMA=h)

  0%|          | 0/256 [00:00<?, ?it/s]